In [12]:
from pycorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP('http://localhost:9000')

In [88]:
import numpy as np
import time
import word2vec
import string
import cPickle as pickle

In [14]:
in_data = None
with open('data/news_reuters_5.csv', 'r') as infile:
    in_data = infile.read().split('\n')
print len(in_data)

16787


In [15]:
title_data = []
for article in in_data:
    fields = article.split(',')
    if len(fields) < 4:
        continue
    title_data += [(fields[0], fields[2], fields[3])]
#     if len(title_data) >= 10:
#         break

In [29]:
print title_data[0]

('GOOG', '20171107', "Alphabet's Waymo to launch robotaxis with no human in driver's seat ")


In [56]:
titles = []
for t in title_data:
    title = t[2].strip()
    title = ''.join(i for i in title if ord(i)<128)
    title = title.replace('  ', ' ')
    title = title.replace('\'s', '')
    for i in range(3):
        if '-' in title and (title[title.find('-') - 1].isupper() or 'UPDATE' in title):
#             print 'REEE'
#             print title
#             print title[title.find('-') + 1:]
            title = title[title.find('-') + 1:]
        else:
            break
    
    title = title.translate(None, string.punctuation)
    title = title + '.'
    title = title.lower()
    if len(titles) > 0 and title in titles[-1][2]:
        continue
    titles += [(t[0], t[1], title)]
#     if len(titles) >= 10:
#         break
print titles[:10]
print len(titles)

[('GOOG', '20171107', 'alphabet waymo to launch robotaxis with no human in driver seat.'), ('GOOG', '20171107', 'indonesia to summon messenger search engine providers over content.'), ('GOOG', '20171102', 'autonation announces waymo fleet repair deal shares jump.'), ('GOOG', '20171102', 'us lawmakers release sample of russianbought facebook ads.'), ('GOOG', '20171031', 'google ditched autopilot driving feature after test user napped behind wheel.'), ('GOOG', '20171027', 'alphabet mobile ad revenue surges shares jump.'), ('GOOG', '20171027', 'no end in sight for tech giant share gains.'), ('GOOG', '20171026', 'alphabet posts qtrly earnings per share of 957.'), ('GOOG', '20171026', 'alphabet revenue rises 24 pct on mobile advertising growth.'), ('GOOG', '20171026', 'alphabet looks to snowy michigan to test selfdriving cars.')]
12680


In [58]:
text = '. '.join([t[2] for t in titles])
print len(text)
print text[:1000]

756721
alphabet waymo to launch robotaxis with no human in driver seat.. indonesia to summon messenger search engine providers over content.. autonation announces waymo fleet repair deal shares jump.. us lawmakers release sample of russianbought facebook ads.. google ditched autopilot driving feature after test user napped behind wheel.. alphabet mobile ad revenue surges shares jump.. no end in sight for tech giant share gains.. alphabet posts qtrly earnings per share of 957.. alphabet revenue rises 24 pct on mobile advertising growth.. alphabet looks to snowy michigan to test selfdriving cars.. alphabet balloon project to provide limited internet in puerto rico.. alphabet capitalg leads lyft 1 billion funding round.. new york times business news  oct 20.. alphabet capitalg leads lyft 1 bln funding round.. lyft says alphabet leads latest 1 billion round of funding.. alphabet to develop hightech waterfront site in toronto.. google launches advanced gmail security features for highrisk u

In [59]:
with open('data/texts/text.txt', 'w') as outfile:
    outfile.write(text)
word2vec.word2phrase('data/texts/text.txt', 'data/texts/text-phrases.txt', verbose=True)

Starting training using file data/texts/text.txt
Words processed: 100K     Vocab size: 59K  
Vocab size (unigrams + bigrams): 37586
Words in train file: 117829


In [81]:
texts = ['']
title2info = {}
index = 0
i = 0
# with open('data/texts/text-phrases.txt', 'r') as infile:
with open('data/texts/text.txt', 'r') as infile:
    for t in infile.read().split('.. '):
        if len(texts[index]) + len(t) > 1e4:
            index += 1
            texts += ['']
        texts[index] += t + ". "
        title2info[t] = (titles[i])
        i += 1
print len(texts)
print title2info.items()[:3]
print len(title2info), len(titles), len(titles) - len(title2info)

75
[('qualcomm talks up future toptier smartphone chip', ('QCOM', '20140407', 'qualcomm talks up future toptier smartphone chip.')), ('paulson  co inc takes share stake in dish network monsanto', ('GOOGL', '20170515', 'paulson  co inc takes share stake in dish network monsanto.')), ('intel pledges 125 mln for startups that back women minorities', ('INTC', '20150609', 'intel pledges 125 mln for startups that back women minorities.'))]
9921 12680 2759


In [61]:
print texts[0]

alphabet waymo to launch robotaxis with no human in driver seat. indonesia to summon messenger search engine providers over content. autonation announces waymo fleet repair deal shares jump. us lawmakers release sample of russianbought facebook ads. google ditched autopilot driving feature after test user napped behind wheel. alphabet mobile ad revenue surges shares jump. no end in sight for tech giant share gains. alphabet posts qtrly earnings per share of 957. alphabet revenue rises 24 pct on mobile advertising growth. alphabet looks to snowy michigan to test selfdriving cars. alphabet balloon project to provide limited internet in puerto rico. alphabet capitalg leads lyft 1 billion funding round. new york times business news  oct 20. alphabet capitalg leads lyft 1 bln funding round. lyft says alphabet leads latest 1 billion round of funding. alphabet to develop hightech waterfront site in toronto. google launches advanced gmail security features for highrisk users. fidelity tests vi

In [62]:
start_time = time.time()

relations = [0 for i in range(100)]
outputs = []
for i in range(len(texts)):
    output = nlp.annotate(texts[i], properties={
      'annotators': 'openie',
      'outputFormat': 'json'
      })
    
    outputs += [output]
    # print output
    print len(output['sentences'])
    for j in range(len(output['sentences'])):
        relations[len(output['sentences'][j]['openie'])] += 1
    print 'text {}: {} sec'.format(i+1, time.time() - start_time)

print relations

158
text 1: 4.24148607254 sec
164
text 2: 5.66943216324 sec
170
text 3: 6.92671108246 sec
170
text 4: 8.197701931 sec
164
text 5: 9.51771593094 sec
164
text 6: 10.8585021496 sec
168
text 7: 12.0351531506 sec
161
text 8: 13.1736140251 sec
159
text 9: 14.3522999287 sec
171
text 10: 15.611634016 sec
169
text 11: 16.7311079502 sec
165
text 12: 17.8191490173 sec
164
text 13: 18.983289957 sec
166
text 14: 20.125369072 sec
165
text 15: 21.3090319633 sec
155
text 16: 22.7500290871 sec
161
text 17: 24.0933060646 sec
165
text 18: 25.4726951122 sec
159
text 19: 27.0111761093 sec
167
text 20: 28.3482151031 sec
169
text 21: 29.77146101 sec
168
text 22: 30.9236240387 sec
170
text 23: 32.0858390331 sec
175
text 24: 33.4842710495 sec
179
text 25: 34.6698861122 sec
176
text 26: 35.9240169525 sec
167
text 27: 37.0436301231 sec
175
text 28: 38.3739719391 sec
165
text 29: 39.5028150082 sec
157
text 30: 40.6110880375 sec
155
text 31: 41.7659840584 sec
170
text 32: 42.9389331341 sec
176
text 33: 44.16612815

In [65]:
print sum(relations[1:]), sum(relations)

8458 12590


In [69]:
print outputs[0]['sentences'][3]['openie']

[{u'subjectSpan': [0, 1], u'relationSpan': [2, 3], u'objectSpan': [3, 4], u'object': u'sample', u'relation': u'release', u'subject': u'us'}, {u'subjectSpan': [0, 1], u'relationSpan': [2, 3], u'objectSpan': [3, 8], u'object': u'sample of russianbought facebook ads', u'relation': u'release', u'subject': u'us'}, {u'subjectSpan': [1, 2], u'relationSpan': [2, 3], u'objectSpan': [3, 8], u'object': u'sample of russianbought facebook ads', u'relation': u'release', u'subject': u'lawmakers'}, {u'subjectSpan': [1, 2], u'relationSpan': [2, 3], u'objectSpan': [3, 4], u'object': u'sample', u'relation': u'release', u'subject': u'lawmakers'}]


In [70]:
print sum([len(outputs[i]['sentences']) for i in range(len(outputs))])

12590


In [89]:
data = {}
bad = 0
good = 0
weird = 0
for o in outputs:
    for s in o['sentences']:
        if len(s['openie']) > 0:
            try:
                info = title2info[' '.join([s['tokens'][i]['word'] for i in range(len(s['tokens']))][:-1])]
    #             print s['openie']
                if info in data:
                    weird += 1
                data[info] = s['openie']
                good += 1
            except:
                print ' '.join([s['tokens'][i]['word'] for i in range(len(s['tokens']))])
                bad += 1
                print 'hmmmm'
#                 print info
#                 print s['openie']
print len(data), bad, good, weird

with open('openie.p', 'w') as outfile:
    pickle.dump(data, outfile)

amazon working on mart glasses as its first wearable device ft. amazon working on mart glasses as its first wearable device ft. alphabet talks with lyft about possible investment source .
hmmmm
third point ups share stake in facebook dissolves in snap inc. google cancels staff meeting over fears of online harassment .
hmmmm
alphabet google announces 50 mln initiative to help people prepare for the changing nature of work blog .
hmmmm
softbank to buy robotics businesses from alphabet inc. softbank unit agrees to buy boston dynamics from alphabet inc. softbank announces agreement to acquire boston dynamics .
hmmmm
exclusive google faces hefty eu fine in shopping case by august sources .
hmmmm
paulson co inc takes share stake in dish network monsanto .
hmmmm
omega advisors ups share stake in alphabet facebook zynga time inc. third point cuts share stake in facebook jpmorgan ups in alphabet .
hmmmm
soros fund management takes share stake in alphabet dissolves stake in gm sec filing .
hmmmm